In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
xl = pd.ExcelFile("./raw_data/NWM_data_updated_land2.xlsx", engine='openpyxl')
a=xl.sheet_names


sheet_no = list(range(2, 32))

In [2]:
#loading the data set
dataframe_collection = {}
i=0
for i in range(33):
    if i==0 :
        dataframe_collection[i] = xl.parse(a[i],converters={"STAID_1":str})
    else:
        dataframe_collection[i] = xl.parse(a[i])


dataframe_collection[0]=dataframe_collection[0].dropna().reset_index(drop=True)

In [3]:
dataframe_collection[0]

,STAID_1,Dist2Sea(degree),LandUse,SoilTexture-1layer,SoilTexture-2layer,SoilTexture-3layer,SoilTexture-4layer,SoilTexture-5layer,SoilTexture-6layer,SoilTexture-7layer,SoilTexture-8layer,SoilTexture-9layer,SoilTexture-10layer,TopographyIndex,DrainageArea(sqml)
0,02315920,1.629911,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,15.0,8.697107,537.0
1,02316000,1.506092,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,15.0,8.778268,550.0
2,023177483,1.081322,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,15.0,9.621850,184.4
3,02317755,1.009225,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,15.0,9.606902,129.0
4,02317797,1.434582,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,15.0,8.694674,577.0
5,02318000,1.141165,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,15.0,8.828504,776.0
6,02318380,1.040585,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,7.0,7.0,15.0,8.986741,1480.0
7,02318500,0.862225,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,7.0,7.0,15.0,9.260724,260.0
8,02327355,0.895303,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,7.0,7.0,8.931155,17.3
9,02327500,0.768052,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,15.0,8.944255,153.0


In [4]:

x_mean = pd.read_csv("./raw_data/x_mean.csv", sep='\t')
x_std =  pd.read_csv("./raw_data/x_std.csv", sep='\t')
y_mean =  pd.read_csv("./raw_data/y_mean.csv", sep='\t')
y_std =  pd.read_csv("./raw_data/y_std.csv", sep='\t')

In [5]:
from keras.models import load_model
import statistics
import math
pred = [None] * 30
std = [None] * 30
std_err = [None] *30
obsflow = dataframe_collection[2]
for l in range(0,30) :
     
    model = load_model('./temporal_DLmodels/my_model_%d_day.h5'%(l))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    modelflow= dataframe_collection[l+3]
    temp =  [0] * 23
    for index, row in dataframe_collection[0].iterrows():
        column_names = ["dis2sea","landUse","soilTexture1", "soilTexture2","soilTexture3","soilTexture4","soilTexture5","soilTexture6","soilTexture7","soilTexture8","soilTexture9","soilTexture10","topoIndex","drainageArea","ModelFlow","Obsflow"]
        df = pd.DataFrame(columns = column_names)  
        k=0
        d=0
      
        sta_id = dataframe_collection[0].iloc[index,0]
        dis2sea=dataframe_collection[0].iloc[index,1]
        land_use=dataframe_collection[0].iloc[index,2]
        soil_texture1=dataframe_collection[0].iloc[index,3]
        soil_texture2=dataframe_collection[0].iloc[index,4]
        soil_texture3=dataframe_collection[0].iloc[index,5]
        soil_texture4=dataframe_collection[0].iloc[index,6]
        soil_texture5=dataframe_collection[0].iloc[index,7]
        soil_texture6=dataframe_collection[0].iloc[index,8]
        soil_texture7=dataframe_collection[0].iloc[index,9]
        soil_texture8=dataframe_collection[0].iloc[index,10]
        soil_texture9=dataframe_collection[0].iloc[index,11]
        soil_texture10=dataframe_collection[0].iloc[index,12]
        topoindex = dataframe_collection[0].iloc[index,13]
        drainagearea =  dataframe_collection[0].iloc[index,14]
        obs = obsflow[sta_id]/drainagearea
        mod = modelflow[sta_id]/drainagearea

        for i in range(481,979):
            df.loc[k]=[dis2sea]+[land_use]+[soil_texture1]+[soil_texture2]+[soil_texture3]+[soil_texture4]+[soil_texture5]+[soil_texture6]+[soil_texture7]+[soil_texture8]+[soil_texture9]+[soil_texture10]+[topoindex]+[drainagearea]+[mod[i]]+[obs[i]]
            k=k+1
        df =df.dropna()
            
        if df.empty:
            temp[index] = 0
            d=d+1
            print(index)
        else :
            x, y = df.iloc[ :, 0:15], df.iloc[:,15]
            x-= x_mean.iloc[l,1:16]
            x /= x_std.iloc[l,1:16]
            
            z=model.predict(x)
            
            x=x*x_std.iloc[l,1:16]
            x=x +x_mean.iloc[l,1:16]
            
            z-= y_mean.iloc[l,1]
            z /= y_std.iloc[l,1]
            
            z=z.ravel()
            z=abs(z)
            a=x.iloc[:,14]
            b=a-(1*z)
            c=a+(z*1)
            res=y.between(b,c)
            count = np.count_nonzero(res)
            temp[index] = count/len(y)   
       
    std[l]= statistics.stdev(temp)    
    pred[l] = sum(temp)/(23-d) 
    std_err[l] = (std[l]*2)/math.sqrt(23-d)
    del model


3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12
3
12


In [6]:
d_x= pd.DataFrame(pred)
d_y=pd.DataFrame(std)
d_z=pd.DataFrame(std_err)
d_xy=pd.concat([d_x, d_y, d_z], axis=1)
d_xy.columns = ['Pred', 'Std', 'Std_err']
d_xy
d_xy.to_csv("./graph_data/prediction_std_stderr_probability_lnd2.csv")